# The whole dataset with Dask

Let us use the Jupyter Lab Dask Extension to create a cluster and watch its metrics

In [18]:
# Your cluster here

In [ ]:
import pandas as pd
import dask.dataframe as dd

In [ ]:
data = dd.read_csv("data/votes_*.csv",
                   usecols=["region", "vote"],
                   dtype={"region": "category",
                          "vote": "category"})

In [ ]:
data

In [ ]:
data.shape

In [ ]:
data.shape[0].compute()

# EVERY VOTE COUNTS

In [ ]:
data["result"] = 1

In [ ]:
count_per_region = (
    data
    .groupby(["region", "vote"])
    .result.agg("count")
    .reset_index()
)

In [ ]:
count_per_region

This is a crucial point: the moment where data has been sufficiently aggregated that it fits in single machine memory.

In [ ]:
count_per_region_computed = count_per_region.compute()

In [ ]:
count_per_region_computed.head()

In [ ]:
def get_winner(count_per_region: pd.DataFrame) -> pd.Series:
    results = list()
    for region, df in count_per_region.groupby("region"):
        results.append(
            {"region": region,
             "winner": df.set_index("vote").result.idxmax()}
        )

    winner_per_region = pd.DataFrame(results)
    delegates_per_region = pd.read_csv("data/region_delegates.csv")
    winner_region_delegates = pd.merge(winner_per_region, delegates_per_region, on="region")
    return (winner_region_delegates
            .groupby("winner")
            .delegates.sum()
            .sort_values(ascending=False))    

In [ ]:
get_winner(count_per_region_computed)

distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
_GatheringFuture exception was never retrieved
future: <_GatheringFuture finished exception=CancelledError()>
asyncio.exceptions.CancelledError
